# Neural Networks
Neural networks can be constructed using the __torch.nn__ package.

Now that we had a glimpse of __autograd__, __nn__ depends on __autograd__ to define models and differentiate them. An __nn.Module__ contains layers, and a method __forward(input)__ that returns the __output__.

A typical training procedure for a neural network is as follows:
* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: __weight = weight - learning_rate * gradient__

### Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1,
        # padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # torch.nn.Linear(in_features, out_features, bias=True)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        # torch.nn.functional.max_pool2d(*args, **kwargs)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square we can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


We just have to define the __forward__ function, and the __backward__ function (where gradients are computed) is automatically defined for us using __autograd__. We can use any of the Tensor operations in the __forward__ function.

The learnable parameters of a model are returned by __net.parameters()__.

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, we should resize the images from the dataset to 32x32.

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0119, -0.0707,  0.0570,  0.2027,  0.1781,  0.0808,  0.0502,  0.1831,
         -0.0795,  0.1425]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

__Recap__
* __torch.Tensor__ - A multi-dimensional array with support for autograd operations like __backward()__. Also holds the gradient w.r.t. the tensor.
* __nn.Module__ - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
* __nn.Parameter__ - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a __Module__.
* __autograd.Function__ - Implements forward and backward definitions of an autograd operation. Every __Tensor__ operation creates at least a single __Function__ node that connects to functions that created a __Tensor__ and encodes its history.

__At this point, we covered__
* Defining a neural network
* Processing inputs and calling backward

__Still Left__
* Computing the loss
* Updating the weights of the network

### Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

In [5]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0402, grad_fn=<MseLossBackward>)


Now, if we follow loss in the backward direction, using its __.grad_fn__ attribute, we will see a graph of computations that looks like this:
> input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
      
So, when we call __loss.backward()__, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has __requires_grad=True__ will have their __.grad__ Tensor accumulated with the gradient.

In [6]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### Backprop
To backpropagate the error all we have to do is to __loss.backward()__. We need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call __loss.backward()__, and have a look at conv1’s bias gradients before and after the backward.

In [7]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([0.0077, 0.0051, 0.0086, 0.0180, 0.0178, 0.0030])


### Update the weights

In [8]:
learning_rate = 1e-2
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as we use neural networks, we want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, there is a small package: __torch.optim__ that implements all these methods.

In [9]:
import torch.optim as optim

# create the optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-2)

# in the training loop
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update